<a href="https://colab.research.google.com/github/450586509/practical-ml/blob/master/bert_sentence_similarity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers==3.3.1

In [2]:
#!pip install transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OneHotEncoder
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
import torch
import transformers as tfs
import warnings
import torch.utils.data as Data
from sklearn import preprocessing
from sklearn.utils import shuffle

warnings.filterwarnings('ignore')

In [ ]:
!curl -LO http://129.204.205.246/similarity_train.csv
!curl -LO http://129.204.205.246/similarity_test.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13.9M  100 13.9M    0     0   117k      0  0:02:01  0:02:01 --:--:--  125k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1577k  100 1577k    0     0  60396      0  0:00:26  0:00:26 --:--:-- 78951


In [ ]:
test_file = "./similarity_test.csv"
train_file="./similarity_train.csv"
train_df = pd.read_csv(train_file, delimiter='\t', header=None)
train_df.columns = ["event_name","text1","text2","label"]
test_df = pd.read_csv(test_file, delimiter='\t', header=None)
test_df.columns = ["event_name","text1","text2","label"]

In [ ]:
train_df.head()

,event_name,text1,text2,label
0,重大合同,公司第四届董事会第十二次临时会议审议通过《关于公司之全资子公司拟与关连人士签订施工合同协议的议案》,许月红获委任爲该公司副总经理,0
1,业绩利好,苏博特：上半年净利润同比预增20%到30%苏博特（603916）7月3日晚间公告,A股异动丨天味食品涨停上半年净利预增82.85%到101.18%,0
2,新增产能,週期底部扩产弹性最大,华侨城通过回购专用证券账户,0
3,公司订单,通达股份先后两次发布预中标国家电网项目提示性公告,中标金额约为人民币1.73亿元,1
4,股权激励,《多氟多化工股份有限公司2020年限制性股票激励计划(草案)》规定的首次授予条件已成就,格隆汇7月28日丨厦门信达披露2020年限制性股票激励计划(草案),1


### 合并两列

In [ ]:
bert_sep = "[SEP]"
train_df["text"] = train_df["text1"] + bert_sep + train_df["text2"]
test_df["text"] = test_df["text1"] + bert_sep + test_df["text2"]

### 新增len列

In [ ]:
print(train_df.shape)
train_df["len"] = train_df.text.apply((lambda x: len(x)))
test_df["len"] = test_df.text.apply((lambda x: len(x)))
print(train_df.shape)
#print(train_df.head)
for index, row in train_df.iterrows():
  print(f"""index={index} text={row.get("text")} len={len(row.get("text"))}""")
  if index > 5:
    break

(83160, 5)
(83160, 6)
index=0 text=公司第四届董事会第十二次临时会议审议通过《关于公司之全资子公司拟与关连人士签订施工合同协议的议案》[SEP]许月红获委任爲该公司副总经理 len=68
index=1 text=苏博特：上半年净利润同比预增20%到30%苏博特（603916）7月3日晚间公告[SEP]A股异动丨天味食品涨停上半年净利预增82.85%到101.18% len=77
index=2 text=週期底部扩产弹性最大[SEP]华侨城通过回购专用证券账户 len=28
index=3 text=通达股份先后两次发布预中标国家电网项目提示性公告[SEP]中标金额约为人民币1.73亿元 len=44
index=4 text=《多氟多化工股份有限公司2020年限制性股票激励计划(草案)》规定的首次授予条件已成就[SEP]格隆汇7月28日丨厦门信达披露2020年限制性股票激励计划(草案) len=81
index=5 text=公司所属部分子公司拟与宁夏建材所属赛马物联科技(宁夏)有限公司(以下简称“赛马物联”)、乌海市西水水泥有限责任公司(以下简称“乌海西水”)签署网络货运服务及熟料采购合同[SEP]电动车企业宝马、奔驰、特斯拉等为保证供应链安全纷纷签订合同 len=118
index=6 text=评级「买入」[SEP]重申中国中铁“买入”评级 len=23


In [ ]:
train_df = train_df[train_df.len < 500]
test_df = test_df[test_df.len < 500]
print(f"""max_len={train_df["len"].max()}""")
print(f"""mean_len={train_df["len"].mean()}""")
print(f"shape={train_df.shape}")
print(train_df.head())
print(test_df.head())

max_len=199
mean_len=63.958585858585856
shape=(83160, 6)
  event_name  ... len
0       重大合同  ...  68
1       业绩利好  ...  77
2       新增产能  ...  28
3       公司订单  ...  44
4       股权激励  ...  81

[5 rows x 6 columns]
  event_name  ... len
0       股份回购  ...  67
1       增持评级  ...  97
2       高管任职  ...  25
3     产品销量上升  ...  47
4       公司订单  ...  87

[5 rows x 6 columns]


In [ ]:
event_counts = train_df.event_name.value_counts()
print(f"type of event_counts = {type(event_counts)}")
print(event_counts)
print(f"样本最少为{event_counts.min()}")
print(f"样本最多为{event_counts.max()}")
print(f"样本平均为{round(event_counts.mean(),4)}")

type of event_counts = <class 'pandas.core.series.Series'>
业绩预增      4532
业绩利好      4525
股市利好      4520
买入评级      4517
增持加仓      4517
重大合同      4517
生产投产      4507
高管任职      4504
股份回购      4503
解除质押      4473
推荐评级      4470
股权激励      4468
对外合作      4466
公司订单      4462
增持评级      4460
新增产能      3202
评级上调      2988
收购并购      2710
优于大市      2307
价格上涨      2264
产品销量上升    2248
Name: event_name, dtype: int64
样本最少为2248
样本最多为4532
样本平均为3960.0


In [3]:
model_path = "bert-base-chinese"
tokenizer = tfs.BertTokenizer.from_pretrained(model_path)
train_tokenized = train_df.text.apply((lambda x: tokenizer.encode(x, add_special_tokens=True)))
print(f"train_tokenized type = {type(train_tokenized)}")
train_max_len = 0
for i in train_tokenized.values:
    if len(i) > train_max_len:
        train_max_len = len(i)

NameError: ignored

In [4]:
model_path = "bert-base-chinese"
tokenizer = tfs.BertTokenizer.from_pretrained(model_path)
x="上升[SEP]公司"
#tokenizer.encode_plus(x,)
t1 = "上市公司"
t2 = "重大消息"
max_len=10
x1,x2 = x.split("[SEP]")
inputs = tokenizer.encode_plus(
            t1,
            t2,
            add_special_tokens=True,
            max_length=max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
            )
print(inputs)
#{'input_ids': [101, 677, 2356, 1062, 1385, 102, 7028, 1920, 3867, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

{'input_ids': [101, 677, 2356, 1062, 1385, 102, 7028, 1920, 3867, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [11]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')
text1_list=["评级由“中性”上调至“买入”","自然语言"]
text2_list=["“MDAE”)与贷款人签署了《贷款合同》","公司"]
inputs = tokenizer(
        text1_list,
        text2_list,
        padding=True,
        #truncation=True,
        #max_length=max_len,
        return_token_type_ids=True
    )
print(f"inputs={inputs}")

#encoded_inputs = tokenizer([t1,t1,"好"],[t2,t2,"的"],padding=True,return_token_type_ids=True)
#print(encoded_inputs)

inputs={'input_ids': [[101, 6397, 5277, 4507, 100, 704, 2595, 100, 677, 6444, 5635, 100, 743, 1057, 100, 102, 100, 100, 100, 114, 680, 6587, 3621, 782, 5041, 5392, 749, 517, 6587, 3621, 1394, 1398, 518, 102], [101, 5632, 4197, 6427, 6241, 102, 1062, 1385, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]}


In [ ]:
y = train_df.label.tolist()
test_y = test_df.label.tolist()

x = train_df.text.tolist()
x,y = shuffle(x,y)
test_x = test_df.text.tolist()
for i,text in enumerate(x):
    print(text)
    if i >=5:
        break
for i , text in enumerate(y):
    print(text)
    if i>=5:
        break
x = np.array(x)
test_x = np.array(test_x)
test_y = [[i] for i in test_y]
test_y = np.array(test_y)
y=[[i] for i in y]
y = np.array(y)
print(type(y))
print(y[0:10])
print(f"x shape ={x.shape}")
print(f"y shape ={y.shape}")

上述人士还表示：“中信集团将收购三家股东（招商证券投资有限公司、青岛港（集团）有限公司、青岛地铁金融控股有限公司）股份[SEP]A股异动|密尔克卫涨停上半年净利预增48%-52%
评级为“增持”摩根士丹利目前给予日清食品（01475）目标价8.6港元以及“增持”投资评级[SEP]上半年新增产能370万吨
碧桂园服务和永升生活服务将以40-60％的增速跑赢大市[SEP]美银美林将玖龙纸业（02689）目标价由8.4港元上调至8.9港元、评级由“买入”降至“中性”评级
公司通过集中竞价交易方式实施2020年度回购股份[SEP]该公司于2020年8月11日斥资89万港元回购100万股
赛轮东营工厂高性能半钢子午胎扩建项目预计何时投产？在轮胎行业处在产能过剩、开工率不足背景下[SEP]于2020年7月24日在香港交易所回购30.0万股
公司通过回购专用证券账户[SEP]以威龙股份名义与烟台银行龙口支行、华夏银行烟台龙口支行和兴业银行烟台分行签署了《保证合同》
0
0
0
1
0
0
<class 'numpy.ndarray'>
[[0]
 [0]
 [0]
 [1]
 [0]
 [0]
 [0]
 [1]
 [0]
 [0]]
x shape =(83160,)
y shape =(83160, 1)


In [ ]:
MAX_LEN = train_max_len
print(f"max_len={MAX_LEN}")
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 8
EPOCHS = 1
LEARNING_RATE = 1e-05
#print(f"max_len={MAX_LEN}")

max_len=195


In [ ]:
class_number=1
print(f"class_number={class_number}")
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert = tfs.BertModel.from_pretrained(model_path)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, class_number)
        #self.out_act = nn.Sigmoid()
    
    def forward(self, ids, mask, token_type_ids,return_dict=True):
        outputs= self.bert(ids, attention_mask = mask, token_type_ids = token_type_ids, return_dict=True)
        #print(type(outputs))
        #print(outputs)
        hidden_state = outputs.hidden_states
        class_vec = outputs.pooler_output
        output_2 = self.dropout(class_vec)
        logits = self.linear(output_2)
        #hidden_average = torch.mean(hidden_state, 1)
        #sentence_vec = hidden_average[0]
        return logits,_

model = BERTClass()
model.to(device)

class_number=1


BERTClass(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
#y=ont_hot
#x:文本数据。
class CustomDataset(Dataset):

    def __init__(self, x, y, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.comment_text = x
        self.targets = y
        self.max_len = max_len

    def __len__(self):
        return len(self.comment_text)

    def __getitem__(self, index):
        comment_text = str(self.comment_text[index])
        #comment_text = " ".join(comment_text.split())
        text1, text2 = comment_text.split("[SEP]")
        
        inputs = self.tokenizer.encode_plus(
            text1,
            text2,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            truncation=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'targets': torch.tensor(self.targets[index], dtype=torch.float)
        }

In [ ]:
# Creating the dataset and dataloader for the neural network

train_x = x
train_y = y

print("FULL Dataset: {}".format(x.shape))
print("TRAIN Dataset: {}".format(train_x.shape))
print("TEST Dataset: {}".format(test_x.shape))
print(f"train_y shape={train_y.shape}")
print(f"test_y shape={test_y.shape}")

training_set = CustomDataset(train_x,train_y, tokenizer, MAX_LEN)
testing_set = CustomDataset(test_x, test_y, tokenizer, MAX_LEN)

FULL Dataset: (83160,)
TRAIN Dataset: (83160,)
TEST Dataset: (9240,)
train_y shape=(83160, 1)
test_y shape=(9240, 1)


In [ ]:
import pickle
def save_data(obj, fn):
    with open(fn, 'wb') as f:
        pickle.dump(obj, f)
save_data(train_x, "s_train_x.pkl")
save_data(train_y, "s_train_y.pkl")
save_data(test_x, "s_test_x.pkl")
save_data(test_y, "s_test_y.pkl")

In [ ]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
testing_loader = DataLoader(testing_set, **test_params)

In [ ]:
import time


def train(epoch):
    model.train()
    start = time.time()
    for step,data in enumerate(training_loader, 0):
        
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)
        outputs,_ = model(ids, mask, token_type_ids)
        if step == 0:
            for row in ids:
                print(row)
            #print(f"ids={ids}")
            print(f"mask={mask}")
            #print(f"")

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        if step%50==0:
            print(f'Epoch: {epoch},step={step}, Loss:  {loss.item()}')
            end = time.time()
            print(f"cost={end-start}")
            start = end
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [ ]:
for epoch in range(1):
    train(epoch)

tensor([ 101, 3419, 7384, 3726,  127, 3299, 8131, 3189,  701, 1290, 6004, 6369,
        6858, 1355,  854, 8439, 2399, 7361, 1169, 2595, 5500, 4873, 4080, 1225,
        6369, 1153,  113, 5770, 3428,  114,  102, 3342,  978, 1069, 5815, 1999,
         818,  711, 1062, 1385, 4324, 4989, 7478, 2809, 6121, 5869,  752, 1350,
        1062, 1385, 2144, 3417, 1999, 1447,  833,  510, 5959, 6992, 1999, 1447,
         833, 1350, 2990, 1399, 1999, 1447,  833, 4638, 2768, 1447,  102,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   

In [ ]:
torch.save(model.state_dict(), "./similarity.model")

### 验证

In [ ]:
def validation(epoch):
    model.eval()
    fin_targets=[]
    fin_outputs=[]
    with torch.no_grad():
        for step, data in enumerate(testing_loader, 0):
            print(f"step={step}----------------------")
            #print(type(data['ids']))
            #print(data['ids'])
            ids = data['ids'].to(device, dtype = torch.long)
            mask = data['mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs,_ = model(ids, mask, token_type_ids)
            
            fin_targets.extend(targets.cpu().detach().numpy().tolist())
            fin_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())
            targets = targets.cpu().detach().numpy().tolist()
            outputs = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
            #outputs = np.array(outputs) >= 0.5
            #outputs = get_true_index(outputs)
            #outputs = np.argmax(outputs, axis=-1)
#             targets = np.array(targets) >= 0.5
#             targets = get_true_index(targets)
            #targets = np.argmax(targets, axis=-1)
            print(f"outputs={outputs}")
            print(f"targets={targets}")
            #break
            
    return fin_outputs, fin_targets

In [ ]:
outputs, targets = validation(1)

step=0----------------------
outputs=[[0.00024322867102455348], [0.9468474388122559], [0.0004644528962671757], [0.00024231111456174403], [0.00023934250930324197], [0.997665286064148], [0.00028125697281211615], [0.9973776340484619]]
targets=[[0.0], [1.0], [0.0], [0.0], [0.0], [1.0], [0.0], [1.0]]
step=1----------------------
outputs=[[0.9965315461158752], [0.997822642326355], [0.0003037711139768362], [0.00026455672923475504], [0.0002719515177886933], [0.00029372997232712805], [0.0005003695259802043], [0.0003107254160568118]]
targets=[[1.0], [1.0], [0.0], [0.0], [0.0], [0.0], [0.0], [0.0]]
step=2----------------------
outputs=[[0.9776867628097534], [0.9952541589736938], [0.0002676680451259017], [0.0002328550472157076], [0.9834191799163818], [0.00031864564516581595], [0.000242864407482557], [0.0006122053018771112]]
targets=[[1.0], [1.0], [0.0], [0.0], [1.0], [0.0], [0.0], [0.0]]
step=3----------------------
outputs=[[0.9966630339622498], [0.00025359727442264557], [0.9978469610214233], [0.

In [ ]:
print(len(outputs))
print(len(targets))

9240
9240


In [ ]:
ouputs_new = [1 if i[0]>0.5 else 0 for i in outputs]
targets_new = [int(i[0]) for i in targets]

In [ ]:
result = [1 if i[0]==i[1] else 0 for i in zip(ouputs_new, targets_new)]

In [ ]:
sum(result)/9240

0.9811688311688311

In [ ]:

max_len = 195
def predict(text1,text2):
    inputs = tokenizer.encode_plus(
        text1,
        text2,
        add_special_tokens=True,
        max_length=max_len,
        pad_to_max_length=True,
        truncation=True,
        return_token_type_ids=True
    )
    ids = torch.tensor([inputs['input_ids']])
    ids = ids.to(device, dtype=torch.long)
    mask = torch.tensor([inputs['attention_mask']])
    mask = mask.to(device, dtype=torch.long)
    token_type_ids = torch.tensor([inputs["token_type_ids"]])
    token_type_ids = token_type_ids.to(device, dtype=torch.long)
    outputs,_ = model(ids, mask, token_type_ids)
    #print(f"outputs 1={outputs}")
    outputs = torch.sigmoid(outputs).cpu().detach().numpy().tolist()
    print(f"outputs={outputs}")
    #outputs = np.array(outputs)
    #output_indexes = np.argmax(outputs, axis=-1)
    #index = output_indexes[0]
    #score = outputs[0][index]
    #labels = le.inverse_transform(output_indexes)
    #return labels[0], score

In [ ]:
text1="4月21日：美团农发行江苏省分行战略合作签约仪式"
text2="通过前期京东开元酒店管理股份有限公司的业务"
predict(text1=text1, text2=text2)
predict(text1=text2, text2=text1)


outputs=[[0.0005229475791566074]]
outputs=[[0.0009311088360846043]]


In [ ]:
for step, data in enumerate(testing_loader, 0):
  print(data)
  break

{'ids': tensor([[ 101, 5450, 6598,  ...,    0,    0,    0],
        [ 101, 2400, 5335,  ...,    0,    0,    0],
        [ 101,  852, 5632,  ...,    0,    0,    0],
        ...,
        [ 101, 3419, 7384,  ...,    0,    0,    0],
        [ 101, 1071,  704,  ...,    0,    0,    0],
        [ 101,  100, 1762,  ...,    0,    0,    0]]), 'mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'targets': tensor([[0.],
        [1.],
        [0.],
        [0.],
        [0.],
        [1.],
        [0.],
        [1.]])}
